<a href="https://colab.research.google.com/github/NUELBUNDI/PDS_PROJECT/blob/main/TimeSeriesPaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
from bokeh.io import output_notebook
from bokeh import models, palettes, transform
from bokeh.plotting import figure, show
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import cluster, decomposition, pipeline, preprocessing
import statsmodels
import missingno as mn
import plotly.offline as pyo
import plotly.express as px
import datetime
import plotly.graph_objects as go
from statsmodels.tsa.stattools import adfuller,acf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf ,plot_pacf
from statsmodels.tsa.arima_model import ARIMA
# import pmdarima as pm
import statsmodels.api as sm
from pylab import rcParams




In [ ]:
# Read The Pd

df=pd.read_csv('https://raw.githubusercontent.com/NUELBUNDI/PDS_PROJECT/main/SalesPerMonthWst.csv')
# df=pd.read_csv("https://raw.githubusercontent.com/NUELBUNDI/PDS_PROJECT/main/climate.csv")
df.columns=['date','sales']
df.head(1)

In [ ]:
# airline = pd.read_csv('AirPassengers.csv',
#                        index_col ='Month',
#                        parse_dates = True)
# train ,test = df[155:189] , df[190:]

In [ ]:
df['date']=pd.to_datetime(df['date'])
df.reset_index()
df=df.set_index('date')
ts=df['sales']
ts.head()


In [ ]:
# Create Train and Test Data

train_data=ts['2010-01-02':'2020-01-01']

test_data=ts['2020-01-02':'2021-01-11']

In [ ]:
# plt.figure(figsize= (10,6))
# plt.plot(df)
# plt.xlabel('Years')
# plt.ylabel('No of Air Passengers')
# plt.title('Trend of the Time Series')

In [ ]:
train_data.dtypes

In [ ]:
# sns.set_style('whitegrid')
# plt.figure(figsize=(15,8))
# lp=sns.lineplot('date','sales',data=train)
# lp.legend(loc='upper right')
# plt.title('Monthly Sales',fontsize=20)
# plt.xlabel('Month')
# # plt.ticklabel_format(style='plain')
# plt.ylabel('Sales')
# plt.show()

In [ ]:
train_data.plot(figsize=(10,5))

plt.show()

In [ ]:
decomposition=seasonal_decompose(train_data,freq=1)


trend=decomposition.trend
seasonal=decomposition.seasonal
residual=decomposition.resid

plt.figure(figsize=(14,8))
plt.subplot(411)
plt.plot(train_data, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

### Checking For Stationarity

In [ ]:
def stationarity_test(timeseries):

    # Get rolling statistics for window = 12 i.e. yearly statistics
    rolling_mean = timeseries.rolling(window = 12).mean()
    rolling_std = timeseries.rolling(window = 12).std()
    
    # Plot rolling statistic
    plt.figure(figsize= (10,6))
    plt.xlabel('Years')
    plt.ylabel('No of Air Passengers')    
    plt.title('Stationary Test: Rolling Mean and Standard Deviation')
    plt.plot(timeseries, color= 'blue', label= 'Original')
    plt.plot(rolling_mean, color= 'green', label= 'Rolling Mean')
    plt.plot(rolling_std, color= 'red', label= 'Rolling Std')   
    plt.legend()
    plt.show()

    print('Results of Dickey-Fuller Test')
    df_test = adfuller(timeseries)
    df_output = pd.Series(df_test[0:4], index = ['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in df_test[4].items():
        df_output['Critical Value (%s)' %key] = value
    print(df_output)
    print(f'Inference: The time series is {"non-" if df_test[1]>=0.05 else ""}stationary')

stationarity_test(train_data)

In [ ]:
# ADF TEST for stationarity
from statsmodels.tsa.stattools import adfuller
def checks_stationarity(ts_data):
    result = adfuller(ts_data)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print(f'Inference: The time series is {"non-" if result[1]>=0.05 else ""}stationary')

res=checks_stationarity(train_data)
res

### Make stationary

In [ ]:
# Check the First Order Difference
train_data_diff=train_data.diff().dropna()

stationarity_test(train_data_diff)

In [ ]:
#Determine P and Q by Plotting the ACF AND PACF

def acf_pacf_plot(ts_data):
  plot_pacf(ts_data)
  plot_acf(ts_data)

acf_pacf_plot(train_data_diff)



### AR MODEL

In [ ]:
model = ARIMA(train_data, order=(2, 1, 0))  
results_AR = model.fit(disp= -1)# If disp < 0 convergence information will not be printed
plt.plot(train_data_diff)
plt.plot(results_AR.fittedvalues, color='red')
plt.title('AR Model, RSS: %.4f'% sum((results_AR.fittedvalues - train_data_diff)**2))

### MA MODEL

In [ ]:
model = ARIMA(train_data, order=(0, 1, 1))  
results_AR = model.fit(disp= -1)# If disp < 0 convergence information will not be printed
plt.plot(train_data_diff)
plt.plot(results_AR.fittedvalues, color='red')
plt.title('MA Model, RSS: %.4f'% sum((results_AR.fittedvalues - train_data_diff)**2))

In [ ]:
model = ARIMA(train_data, order=(2, 1, 2))  
results_ARIMA = model.fit(disp=-1)  
plt.plot(train_data_diff)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('Combined Model, RSS: %.4f'% sum((results_ARIMA.fittedvalues-train_data_diff)**2))

In [ ]:
predictions_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)

print('Total no of predictions: ', len(predictions_diff))
predictions_diff.head()

In [ ]:
predictions_diff_cumsum = predictions_diff.cumsum()
predictions_diff_cumsum.head()

In [ ]:


results_ARIMA.plot_predict(start =1, end= 204) 

In [ ]:
forecast_values_log_scale = results_ARIMA.forecast(steps = 60)
forecast_values_original_scale = np.exp(forecast_values_log_scale[0])

forecast_date_range= pd.date_range("1961-01-01", "1965-12-01", freq="MS")

df_forecast =pd.DataFrame(forecast_values_original_scale, columns=['Forecast'])
df_forecast['Month'] = forecast_date_range

df_forecast[['Month', 'Forecast']]

In [ ]:
# TRAIN ARIMA MODEL

order=(2,1,2)
model=ARIMA(endog=train_data,order=order)
fit=model.fit()

In [ ]:
print(fit.summary())

In [ ]:
# Plot residual errors

residuals = pd.DataFrame(fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
# Actual vs Fitted
fit.plot_predict(dynamic=False)
plt.show()

### Build an ARIMA model manually.

In [ ]:
#How to do find the optimal ARIMA model manually using Out-of-Time Cross validation

# Build Model
# model = ARIMA(train, order=(3,2,1))  
# order=(2,1,2)
# model=ARIMA(endog=train_data,order=order)

model=ARIMA(endog=train_data, order=(1, 1, 0))  
fitted = model.fit(disp=-1)  

# Forecast
fc, se, conf = fitted.forecast(22, alpha=0.05)  # 95% conf

# Make as pandas series
fc_series = pd.Series(fc, index=test_data.index)
lower_series = pd.Series(conf[:, 0], index=test_data.index)
upper_series = pd.Series(conf[:, 1], index=test_data.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train_data, label='training')
plt.plot(test_data, label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()


In [ ]:
# Accuracy metrics
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    acf1 = acf(fc-test_data)[1]                      # ACF1
    return({'MAPE':mape, 'ME':me, 'MAE': mae, 
            'MPE': mpe, 'RMSE':rmse, 'ACF1':acf1, 
            'CORR':corr, 'MINMAX':minmax})

summary=forecast_accuracy(fc, test_data.values)

summary


### How to do Auto Arima Forecast in Python

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

model = pm.auto_arima(train_data, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=5, max_q=5, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

In [ ]:
model.plot_diagnostics(figsize=(7,5))
plt.show()

Top left: The residual errors seem to fluctuate around a mean of zero and have a uniform variance.

Top Right: The density plot suggest normal distribution with mean zero.

Bottom left: All the dots should fall perfectly in line with the red line. Any significant deviations would imply the distribution is skewed.

Bottom Right: The Correlogram, aka, ACF plot shows the residual errors are not autocorrelated. Any autocorrelation would imply that there is some pattern in the residual errors which are not explained in the model. So you will need to look for more X’s (predictors) to the model.

Overall, it seems to be a good fit. Let’s forecast.

In [ ]:
len(train_data)

In [ ]:
# Forecast
n_periods = 22
fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
index_of_fc = np.arange(len(train_data), len(train_data)+n_periods)

# make series for plotting purpose
fc_series = pd.Series(fc, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)

# Plot
plt.plot(train_data)
plt.plot(fc_series, color='darkgreen')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.title("Final Forecast of WWW Usage")
# plt.show()

In [ ]:
# https://www.machinelearningplus.com/resources/arima/arima-forecast-test-results/
# https://www.machinelearningplus.com/resources/arima/implement-arima-model/
# https://www.machinelearningplus.com/time-series/time-series-analysis-python/
# https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/
# pip install statsmodels==0.11.0
# https://www.kaggle.com/satishgunjal/tutorial-time-series-analysis-and-forecasting
# https://www.geeksforgeeks.org/python-arima-model-for-time-series-forecasting/